In [757]:
from math import gcd 
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from datetime import datetime

import qiskit
from qiskit import assemble, transpile
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error
import qiskit.ignis.verification.randomized_benchmarking as rb

qiskit.__version__

'0.19.1'

# Library

In [1183]:
def circuit_step_2_qubit_idx(cstep, swap_qubits=False):
    qregister = cstep[1]
    if len(qregister) == 2:
        return [0,1]
    elif len(qregister) == 1:
        reg_str = str(qregister[0])
        if ", 1" in reg_str:
            return [1] if not swap_qubits else [0]
        elif ", 0" in reg_str:
            return [0] if not swap_qubits else [1]
    elif len(qregister) > 2:
        raise NotImplementedError

def circuit_step_2_gatestr(cstep):
    return cstep[0].name, [float(p) for p in cstep[0].params]

def circuit_2_step_list(circ, qubit_idx=0, swap_qubits=False):
    gate_strs, on_nv = [],[]
    for step in circ.data:
         gate_strs.append(circuit_step_2_gatestr(step))
         on_nv.append(circuit_step_2_qubit_idx(step, swap_qubits=swap_qubits))
    
    step_list = list(zip(gate_strs, on_nv))
    #return step_list
    return [step for step in step_list if qubit_idx in step[1]]

def serialize_step_list(steps_q0, steps_q1):
    # todo: seems over-complicated, but works
    ser_list = []
    
    idx_0, idx_1 = 0, 0
    while not (idx_0==len(steps_q0) and idx_1==len(steps_q1)):   
        is_end_0, is_end_1 = False, False
        is_2q_0, is_2q_1 = False, False
        try:
            is_2q_0 = len(steps_q0[idx_0][1]) == 2
        except IndexError:
            is_end_0 = True
        try:
            is_2q_1 = len(steps_q1[idx_1][1]) == 2
        except IndexError:
            is_end_1 = True
        
        #print(f"Now list: {ser_list}. \nEnd: {is_end_0}, {is_end_1}, 2q: {is_2q_0}, {is_2q_1}")
        try:
            if (not is_2q_0) and (not is_2q_1) and (not is_end_0) and (not is_end_1):
                ser_list.append(steps_q0[idx_0])
                ser_list.append(steps_q1[idx_1])
                idx_0 += 1
                idx_1 += 1

            elif (is_2q_0) and (not is_2q_1) and (not is_end_1):
                ser_list.append(steps_q1[idx_1])
                idx_1 += 1

            elif (not is_2q_0) and (is_2q_1) and (not is_end_0):
                ser_list.append(steps_q0[idx_0])
                idx_0 += 1

            elif (is_2q_0) and (is_2q_1):
                ser_list.append(steps_q0[idx_0])
                idx_0 += 1
                idx_1 += 1
            elif is_end_0:
                ser_list.append(steps_q1[idx_1])
                idx_1 += 1
            elif is_end_1:
                ser_list.append(steps_q0[idx_0])
                idx_0 += 1
            else:
                raise RuntimeError(f"Failed at idxs {idx_0}, {idx_1}")
                
        except IndexError:
            raise IndexError(f"Failed at idxs {idx_0}, {idx_1}")
        
            
    return ser_list

def get_n_cx(circ):
    return np.sum([el[0].name=='cx' for el in circ.data])

0

0

# Create random benchmarking list


In [1236]:
params = {}

# number of qubits
params['n_qubits'] = 2


rb_opts = {}
#Number of Cliffords in the sequence
rb_opts['length_vector'] = [i for i in range(1,10)]
#rb_opts['length_vector'] = [2,4,8,16,32,48,64,96,128,144,192, 256]
# Number of seeds (random sequences)
rb_opts['nseeds'] = 10
# Default pattern
rb_opts['rb_pattern'] = [[0,1]] if params['n_qubits']==2 else [[0]]

rb_circs, xdata = rb.randomized_benchmarking_seq(**rb_opts)
# transpose rb_circ st 2nd index is random index
rb_circs = list(map(list, zip(*rb_circs)))

now = datetime.now() # current date and time
params['timestamp'] = now.strftime("%Y%m%d-%H%M-%S")
params['qiskit_version'] = qiskit.__version__
params['rb_options'] = rb_opts

params

{'n_qubits': 2,
 'timestamp': '20230130-0931-35',
 'qiskit_version': '0.19.1',
 'rb_options': {'length_vector': [1, 2, 3, 4, 5, 6, 7, 8, 9],
  'nseeds': 10,
  'rb_pattern': [[0, 1]]}}

In [1237]:
len(rb_circs[0]), xdata

(10, array([[1, 2, 3, 4, 5, 6, 7, 8, 9]]))

In [1238]:
# calculate number of cnots per xdata step
arr = np.zeros((len(rb_circs),len(rb_circs[0])))
for i in range(0, len(rb_circs)):
    for j in range(0, len(rb_circs[0])):
        arr[i,j] = get_n_cx(rb_circs[i][j])
        
np.mean(arr, axis=1)

array([ 2.6,  4.1,  5.3,  7. ,  8.1,  9.5, 11.3, 13.1, 14.5])

In [1239]:
circ = rb_circs[0][9]  # indices: n_order, m_random
circ.draw()


┌───┐┌───┐               ┌─────┐┌───┐      ░       ┌───┐ ┌───┐     »
qr_0: ┤ Y ├┤ S ├────────────■──┤ Sdg ├┤ H ├──■───░───■───┤ H ├─┤ S ├──■──»
      ├───┤├───┤┌───┐┌───┐┌─┴─┐└┬───┬┘├───┤┌─┴─┐ ░ ┌─┴─┐┌┴───┴┐├───┤┌─┴─┐»
qr_1: ┤ Z ├┤ S ├┤ H ├┤ S ├┤ X ├─┤ H ├─┤ S ├┤ X ├─░─┤ X ├┤ Sdg ├┤ H ├┤ X ├»
      └───┘└───┘└───┘└───┘└───┘ └───┘ └───┘└───┘ ░ └───┘└─────┘└───┘└───┘»
cr: 2/═══════════════════════════════════════════════════════════════════»
                                                                         »
«      ┌─────┐┌───┐       ┌─┐        
«qr_0: ┤ Sdg ├┤ Y ├───────┤M├────────
«      ├─────┤├───┤┌─────┐└╥┘┌───┐┌─┐
«qr_1: ┤ Sdg ├┤ H ├┤ Sdg ├─╫─┤ Z ├┤M├
«      └─────┘└───┘└─────┘ ║ └───┘└╥┘
«cr: 2/════════════════════╩═══════╩═
«                          0       1

The defualt gate set used is by qiskit convention (X,Y,Sdg,H,CX). We transpile to a somewhat more useful gate set for NV experiment (1, pi2_x, pi2_y, CNOT).

In [1240]:
import copy as cp

rb_circs_native = []
for circ_list in rb_circs:
    rb_i = []
    for circ_in in circ_list:
        rb_i.append(transpile(circ_in, basis_gates=['id', 'ry', 'rx', 'cx']) )
    rb_circs_native.append(cp.deepcopy(rb_i))

In [1241]:

circ = rb_circs_native[0][0]
circ.draw()


global phase: π
      ┌─────────┐┌──────────┐┌──────────┐     ┌─────────┐┌─────────┐ ░ »
qr_0: ┤ Rx(π/2) ├┤ Ry(-π/2) ├┤ Rx(-π/2) ├──■──┤ Ry(π/2) ├┤ Rx(π/2) ├─░─»
      └┬───────┬┘└──────────┘└──────────┘┌─┴─┐├─────────┤├─────────┤ ░ »
qr_1: ─┤ Rx(π) ├─────────────────────────┤ X ├┤ Ry(π/2) ├┤ Rx(π/2) ├─░─»
       └───────┘                         └───┘└─────────┘└─────────┘ ░ »
cr: 2/═════════════════════════════════════════════════════════════════»
                                                                       »
«      ┌──────────┐┌──────────┐     ┌─────────┐┌─────────┐┌──────────┐┌─┐
«qr_0: ┤ Rx(-π/2) ├┤ Ry(-π/2) ├──■──┤ Rx(π/2) ├┤ Ry(π/2) ├┤ Rx(-π/2) ├┤M├
«      ├──────────┤├──────────┤┌─┴─┐└┬───────┬┘└───┬─┬───┘└──────────┘└╥┘
«qr_1: ┤ Rx(-π/2) ├┤ Ry(-π/2) ├┤ X ├─┤ Rx(π) ├─────┤M├─────────────────╫─
«      └──────────┘└──────────┘└───┘ └───────┘     └╥┘                 ║ 
«cr: 2/═════════════════════════════════════════════╩══════════════════╩═
«                                                   1                  0

In [1055]:
circ.data[0][0].params

[-1.5707963267948966]

## Serialize to non-parallel qubit operations

Don't do 1q operations in parallel, but step by step.

In [703]:
len(steps_q0), len(steps_q1)

(0, 8)

In [1188]:
# chose a circ
idx_order, idx_circ = 1, 1
circ = rb_circs_native[idx_order][idx_circ]

# swap qubit, as qiskit traspiles to c1not2. We want c2not1.
steps_q0, steps_q1 = circuit_2_step_list(circ, qubit_idx=0, swap_qubits=True), circuit_2_step_list(circ, qubit_idx=1, swap_qubits=True)
serialize_step_list(steps_q0, steps_q1)

[(('rx', [1.5707963267948966]), [0]),
 (('rx', [1.5707963267948966]), [1]),
 (('ry', [-1.5707963267948968]), [1]),
 (('cx', []), [0, 1]),
 (('ry', [1.5707963267948966]), [0]),
 (('ry', [1.5707963267948966]), [1]),
 (('rx', [1.5707963267948966]), [0]),
 (('rx', [1.5707963267948966]), [1]),
 (('cx', []), [0, 1]),
 (('barrier', []), [0, 1]),
 (('rx', [1.5707963267948966]), [0]),
 (('ry', [-1.5707963267948968]), [1]),
 (('ry', [-1.5707963267948966]), [0]),
 (('rx', [-1.5707963267948966]), [0]),
 (('cx', []), [0, 1]),
 (('ry', [1.5707963267948966]), [0]),
 (('rx', [1.5707963267948966]), [0]),
 (('barrier', []), [0, 1]),
 (('cx', []), [0, 1]),
 (('ry', [1.5707963267948966]), [0]),
 (('ry', [1.5707963267948966]), [1]),
 (('rx', [1.5707963267948966]), [0]),
 (('rx', [1.5707963267948966]), [1]),
 (('cx', []), [0, 1]),
 (('rx', [-1.5707963267948966]), [0]),
 (('rx', [-1.5707963267948966]), [1]),
 (('ry', [-1.5707963267948966]), [0]),
 (('ry', [-1.5707963267948966]), [1]),
 (('cx', []), [0, 1]),


## Save to file

Iterate over (converted to native gates) gate sequence and create a qudi compatible sequence as a json file.

In [1231]:
qudi_gates = []
idx = 0


n_seqs = len(rb_circs_native[0])*len(rb_circs_native)
for idx_cl, circ_list in enumerate(rb_circs_native):
    circs_i = []
    for idx_r, circ in enumerate(circ_list):
        #circs_i.append()
        # swap qubit, as qiskit traspiles to c1not2. We want c2not1.
        steps_q0 = circuit_2_step_list(circ, qubit_idx=0, swap_qubits=True)
        steps_q1 = circuit_2_step_list(circ, qubit_idx=1, swap_qubits=True)
        
        idx_dict = {"idx_info":{"n_cliff": rb_opts['length_vector'][idx_cl],
                    "idx_cliff": idx_cl,
                    "idx_random": idx_r}}
        params_dict = {"script params": params}
        
        header = f"### Benchmark Sequence {idx+1}/{n_seqs} ###\n"
        header += f"# {json.dumps(idx_dict)}"
        header += f"# {json.dumps(params)}"
        header += f"# Columns: operation | params | idx_qubit\n"

        
        
        steps = [header]
        steps.append(serialize_step_list(steps_q0, steps_q1))
        
        circs_i.append(steps)
        idx += 1
        
    qudi_gates.append(circs_i)
    

In [1232]:
qudi_gates[0][6]

['### Benchmark Sequence 7/120 ###\n# {"idx_info": {"n_cliff": 2, "idx_cliff": 0, "idx_random": 6}}# {"n_qubits": 1, "timestamp": "20230130-0930-16", "qiskit_version": "0.19.1", "rb_options": {"length_vector": [2, 4, 8, 16, 32, 48, 64, 96, 128, 144, 192, 256], "nseeds": 10, "rb_pattern": [[0]]}}# Columns: operation | params | idx_qubit\n',
 [(('ry', [1.5707963267948966]), [1]),
  (('rx', [-1.5707963267948966]), [1]),
  (('barrier', []), [1]),
  (('ry', [-1.5707963267948966]), [1]),
  (('rx', [-3.141592653589793]), [1]),
  (('barrier', []), [1]),
  (('rx', [1.5707963267948966]), [1]),
  (('ry', [1.5707963267948966]), [1]),
  (('rx', [1.5707963267948966]), [1]),
  (('measure', []), [1])]]

Perform the saving

In [1233]:
import json

fname = f"{params['timestamp']}_rb_seq_{params['n_qubits']}q.json"
with open(fname, 'w') as f:
    json.dump(qudi_gates, f, indent=None)
fname

'20230130-0930-16_rb_seq_1q.json'

Load saved file to check.

In [1234]:
with open(fname) as f:
    qudi_gates = json.load(f)
    
qudi_gates[0][0]

['### Benchmark Sequence 1/120 ###\n# {"idx_info": {"n_cliff": 2, "idx_cliff": 0, "idx_random": 0}}# {"n_qubits": 1, "timestamp": "20230130-0930-16", "qiskit_version": "0.19.1", "rb_options": {"length_vector": [2, 4, 8, 16, 32, 48, 64, 96, 128, 144, 192, 256], "nseeds": 10, "rb_pattern": [[0]]}}# Columns: operation | params | idx_qubit\n',
 [[['ry', [-1.5707963267948966]], [1]],
  [['rx', [1.5707963267948966]], [1]],
  [['barrier', []], [1]],
  [['ry', [-1.5707963267948966]], [1]],
  [['rx', [1.5707963267948966]], [1]],
  [['barrier', []], [1]],
  [['ry', [-1.5707963267948966]], [1]],
  [['rx', [1.5707963267948966]], [1]],
  [['measure', []], [1]]]]